<a href="https://colab.research.google.com/github/adamschmideg/ethereum/blob/master/travis/Visualize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
url = "https://raw.githubusercontent.com/adamschmideg/ethereum/master/travis/data/"
builds = pd.read_csv(url + "builds.csv")
jobs = pd.read_csv(url + "jobs.csv")
failures = pd.read_csv(url + "failures.csv")

In [0]:
bj = pd.merge(builds, jobs, left_on="Id", right_on="BuildId", suffixes=("_Build", "_Job"))
full = pd.merge(bj, failures, left_on="Id_Job", right_on="JobId")

Filter rows with a failed job that belong to builds where it was pushed to master

In [0]:
cutoff_date = '2020-03-03'


In [0]:
df = full[['Test', 'PullRequestNumber']].groupby(by='Test').nunique()
not_one_pr = df[df.PullRequestNumber.ne(1)].Test.keys().values


In [22]:
with_tests = full[full.Test.isin(not_one_pr)]
filtered = with_tests[with_tests.State_Job.eq('failed') & with_tests.StartedAt_Build.gt(cutoff_date)]
cols = filtered[['Package', 'Test', 'Id_Job']]
cols.groupby(['Test', 'Package']).count().sort_values(by=['Id_Job'], ascending=False)

,,Id_Job
Test,Package,
TestHandshake,les,7
TestTimerFunc,metrics,6
TestBroadcastBlock,eth,5
TestOdrGetReceiptsLes2,light,4
TestOdrGetBlockLes2,light,4
TestOdrContractCallLes2,light,4
TestOdrAccountsLes2,light,4
TestBoundedForkedSync63Fast,eth/downloader,3
TestSimulation,whisper/whisperv6,3
